### CMPS242 Homework 5 - Classifying Donald Trump and Hillary Clinton's Tweets
#### Team Members:
Alex                       
Karthik Balakrishna           
Roy Shadmon 
Sasidharan Mahalingam


#### Importing the required packages
First import the rerquired packages required to create a neural network

In [33]:
#Importing Packages
import tensorflow as tf
import os
import pandas as pd
import numpy as np

#### Loading Train data
Import the tweets from the given dataset

In [34]:
#TODO: Have to implement the text manupilation to remove the https and other unwanted characters and strings

#Loading the train dataset
df1 = pd.read_csv('train.csv')
#Loading the test dataset
df2 = pd.read_csv('test.csv')
#Getting only the tweets from the train dataset
train_tweets = df1['tweet'].tolist()
#Getting only the tweets from the test dataset
test_tweets = df2['tweet'].tolist()
#Printing a sample of the train dataset tweets
print('Printing a sample of the train dataset tweets... \n\n')
print(train_tweets[0:5])
#Printing a sample of the test dataset tweets
print('Printing a sample of the test dataset tweets... \n\n')
print(test_tweets[0:5])
#Concatenating the train and test dataset tweets to form the word embeddings
tweets = train_tweets + test_tweets

Printing a sample of the train dataset tweets... 


['In Tampa, Florida- thank you to all of our outstanding volunteers who want to #MakeAmericaGreatAgain! https://t.co/04qqpGylT7', 'Poll: @realDonaldTrump vs. @HillaryClinton among white Evangelicals. https://t.co/6ohwIh1Q24', 'Obama on whether Trump could be trusted with US nuclear weapons: "Make your own judgment" https://t.co/6OZtrfIwim https://t.co/Nj20PaXF2o', '"Hillary Clinton has never quit on anything in her life." —@FLOTUS #DemsInPhilly https://t.co/mbg5H8QZA3', 'I LOVE NEW YORK! #NewYorkValues \nhttps://t.co/dbTDhYAX1v']
Printing a sample of the test dataset tweets... 


['Live from Charlotte: @POTUS hits the trail with Hillary for the first time in this campaign. Watch: https://t.co/1IJMGhsqEc', 'The worst part is, Trump\'s disturbing policies for immigrant families go way beyond just "building a wall." https://t.co/szrTF9eB0E', 'You can watch 360 video live from the podium! https://t.co/yqcIsBUdAi #RNCinCLE #TrumpIsWithYou 

#### Tokenizing the tweets
Tokenize the tweets using keras Tokenizer

In [40]:
#Tokenize all the tweets
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(tweets)
#Printing some of the word indexs
sample_mapping = {k: tokenizer.word_index[k] for k in list(tokenizer.word_index)[:10]} 
print('Printing a few word index mappings... \n\n')
print(sample_mapping)

Printing a few word index mappings... 


{'beau': 12252, 'streep': 9770, 'luxurious': 9595, "kurteichenwald's": 11192, '6rh849dlyz': 13901, 'djkdazt3wv': 11083, 'quo': 3621, 'introduction': 3687, 'equality': 897, 'realericjallen': 12351}


#### Loading GloVe word embeddings
Load the downloaded pre-trained GloVe word vectors

In [28]:
#Create an empty dictionary that stores the word to index mappings
embeddings_index = {}
#Import the GloVe word vectors
glove_data = 'glove.twitter.27B.200d.txt'
#Load the word vectors into memory
f = open(glove_data)
for line in f:
    values = line.split()
    word = values[0]
    value = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = value
f.close()

#Printing the number of word vectors loaded
print('Loaded %s word vectors.' % len(embeddings_index))


Loaded 1193514 word vectors.


#### Mapping the words to the GloVe word vectors
Map the tokenized words to 200 dimensional word vectors

In [29]:
#Setting the dimension of the word vectors

#Note: I have used 200 dimensional vectors (I know it might overfit as the given data is very less). 
#Let's zero down on the optimal value after we get this to work

embedding_dimension = 200
#Getting the word index representation of the tokenized words
word_index = tokenizer.word_index

embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimension))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector[:embedding_dimension]

#Printing the shape of the word embedding matrix
print(embedding_matrix.shape)
 
#TODO:    
#One future improvement that we could do is to train these embedding using the tweets instead of setting the words not present to zeros

(14120, 200)


#### Get the tweets from the training dataset and for it's one hot vector representation
Get the training tweet dataset and convert that from strings into a one-hot vector representation, so that we can use the word embeddings to create the required input to the LSTM

In [43]:
#Create a tokenizer to tokenize the train tweets
train_tokenizer = tf.keras.preprocessing.text.Tokenizer()
#Fit the tokenizer on the train dataset
train_tokenizer.fit_on_texts(train_tweets)
#Creating the one-hot vector representation of the train tweets
train_one_hot = np.array(train_tokenizer.texts_to_matrix(train_tweets)).astype(int)
print(train_one_hot)

[[0 1 1 ..., 0 0 0]
 [0 1 1 ..., 0 0 0]
 [0 1 1 ..., 0 0 0]
 ..., 
 [0 1 1 ..., 0 0 0]
 [0 1 1 ..., 1 0 0]
 [0 1 1 ..., 0 1 1]]


#### Convert to one-hot vectors into matrices using the word embeddings
Now that we have the one-hot vector representation of the tweets, use the created GloVe word vectors to generate the input dataset that has the word vectors instead of the one-hot vectors

In [47]:
#Now create the required input to our LSTM using the embedding and the one-hot vectors
embed = tf.nn.embedding_lookup(embedding_matrix, train_one_hot)
#Printing the dimensions of the input dataset
print('The shape of the input dataset is:')
print(embed.shape)

The shape of the input dataset is:
(5000, 11939, 200)
